In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
from pascal_voc_writer import Writer


#cap = cv.VideoCapture(r"C:\Users\admin\Downloads\Top 10 Safety Vest For Construction For Men And Women.mp4")
whT = 320
confThreshold =0.5
nmsThreshold= 0.2
 
#### LOAD MODEL
## Coco Names
classesFile = r"C:\Users\admin\Downloads\wobot\obj.names"
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)
## Model Files
modelConfiguration = r"C:\Users\admin\Downloads\wobot\yolov4-tiny-custom.cfg"
modelWeights = r"C:\Users\admin\Downloads\wobot\yolov4-tiny-custom_best.weights"
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)



def findObjects(outputs,img,file):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]*wT) , int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2) , int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
    
    indices = cv.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    writer = Writer(file, hT,wT)
    basename = os.path.basename(file)
    basename_no_ext = os.path.splitext(basename)[0]
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                  (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        a=classNames[classIds[i]]
        writer.addObject(a, x, y, x+w,y+h)
    writer.save(output_path + basename_no_ext + '.xml')
            
        
        


import os
folder=r"C:\Users\admin\Desktop\Images"
output_path=r"C:\Users\admin\Desktop\XML\ "
for dest in os.listdir(folder):
    file=os.path.join(folder,dest)
    img=cv.imread(file)
    blob = cv.dnn.blobFromImage(img, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    findObjects(outputs,img,file)




['helmet', 'head']
